In [1]:
# Data Integration Activity
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import seaborn

# Aggregate Census Data
census_data = pd.read_csv('acs2017_census_tract_data.csv')
print(census_data.head())
census_data = pd.concat([census_data["State"], census_data["County"], census_data["TotalPop"], census_data["Income"], census_data["IncomePerCap"], census_data["Poverty"]],axis=1)
census_data.head()

      TractId    State          County  TotalPop   Men  Women  Hispanic  \
0  1001020100  Alabama  Autauga County      1845   899    946       2.4   
1  1001020200  Alabama  Autauga County      2172  1167   1005       1.1   
2  1001020300  Alabama  Autauga County      3385  1533   1852       8.0   
3  1001020400  Alabama  Autauga County      4267  2001   2266       9.6   
4  1001020500  Alabama  Autauga County      9965  5054   4911       0.9   

   White  Black  Native  ...  Walk  OtherTransp  WorkAtHome  MeanCommute  \
0   86.3    5.2     0.0  ...   0.5          0.0         2.1         24.5   
1   41.6   54.5     0.0  ...   0.0          0.5         0.0         22.2   
2   61.4   26.5     0.6  ...   1.0          0.8         1.5         23.1   
3   80.3    7.1     0.5  ...   1.5          2.9         2.1         25.9   
4   77.5   16.4     0.0  ...   0.8          0.3         0.7         21.0   

   Employed  PrivateWork  PublicWork  SelfEmployed  FamilyWork  Unemployment  
0       881  

,State,County,TotalPop,Income,IncomePerCap,Poverty
0,Alabama,Autauga County,1845,67826.0,33018.0,10.7
1,Alabama,Autauga County,2172,41287.0,18996.0,22.4
2,Alabama,Autauga County,3385,46806.0,21236.0,14.7
3,Alabama,Autauga County,4267,55895.0,28068.0,2.3
4,Alabama,Autauga County,9965,68143.0,36905.0,12.2


In [2]:
# Aggregate Covid Data
covid_data = pd.read_csv('COVID_county_data.csv')
print(covid_data.head())
covid_data["date"] = pd.to_datetime(covid_data["date"])
df1 = covid_data.drop(labels="fips",axis=1)
df2 = df1.groupby(["state", "county"]).sum()
df2

         date     county       state     fips  cases  deaths
0  2020-01-21  Snohomish  Washington  53061.0      1     0.0
1  2020-01-22  Snohomish  Washington  53061.0      1     0.0
2  2020-01-23  Snohomish  Washington  53061.0      1     0.0
3  2020-01-24       Cook    Illinois  17031.0      1     0.0
4  2020-01-24  Snohomish  Washington  53061.0      1     0.0


cases   deaths
state   county                    
Alabama Autauga    645935   9042.0
        Baldwin   2003567  23041.0
        Barbour    268771   4077.0
        Bibb       261043   5272.0
        Blount     630106   8669.0
...                   ...      ...
Wyoming Teton      305376    617.0
        Uinta      200783   1037.0
        Unknown        37      0.0
        Washakie    84354   2622.0
        Weston      56976    286.0

[3274 rows x 2 columns]

In [3]:
# Derive totals for December 2020
start = datetime.datetime(2020, 12, 1)
end = datetime.datetime(2020, 12, 31)
df3 = df1[df1["date"].isin(pd.date_range(start,end))]
df4 = df3.groupby(["state", "county"]).sum()
df4.columns = ["dec2020_cases", "dec2020_deaths"]

In [4]:
# Dataframe for covid aggregate data
covid_agg = df2.join(df4)
covid_agg

# Sample results from some counties
sample = [("Virginia", "Loudoun"), ("Oregon", "Washington"), ("Kentucky", "Harlan"), ("Oregon", "Malheur")]
df5 = []
for s,c in sample:
    df5.append(covid_agg.loc[s].loc[c])
df5 = pd.DataFrame(df5)
df5

,cases,deaths,dec2020_cases,dec2020_deaths
Loudoun,2496450.0,35820.0,376223.0,4729.0
Washington,2157339.0,22455.0,424620.0,3860.0
Harlan,205984.0,3994.0,38959.0,506.0
Malheur,453634.0,7770.0,82916.0,1465.0


In [5]:
# Generate raw numbers for poverty rate per county 
census_data = census_data.assign(pov_nos=census_data["Poverty"] * census_data["TotalPop"] / 100)
census_data
df8 = census_data.groupby(["State", "County"]).sum()

In [6]:
# Generate Poverty percent of total population  by county
df8["Poverty"] = (df8["pov_nos"] * 100)/  df8["TotalPop"]
# Generate Income per capital per county by dividing the total income of the county by its total poopulation
df8["IncomePerCap"] = (df8["Income"] / df8["TotalPop"])
df8

# Dataframe for census aggregate data
census_agg = df8.drop(labels="pov_nos", axis=1)

In [7]:
# Sample results from some counties
sample = [("Virginia", "Loudoun County"), ("Oregon", "Washington County"), ("Kentucky", "Harlan County"), ("Oregon", "Malheur County")]
df5 = []
for s,c in sample:
    df5.append(census_agg.loc[s].loc[c])
df5 = pd.DataFrame(df5)
df5

,TotalPop,Income,IncomePerCap,Poverty
Loudoun County,374558.0,8298861.0,22.156411,3.689598
Washington County,572071.0,7961909.0,13.917694,10.321202
Harlan County,27548.0,291194.0,10.570423,35.669482
Malheur County,30421.0,272162.0,8.946517,24.298225


In [8]:
census_agg.reset_index(level=['State','County'],inplace=True)
covid_agg.reset_index(level=['state','county'], inplace=True)

In [9]:
# Prep for integration of census_agg and covid_agg
s1 = set(covid_agg["state"].unique())
s2 = set(census_agg["State"].unique())
print(s1 -s2)
census_agg["County"] = census_agg["County"].str[:-7]
census_agg[census_agg["State"] == "Puerto Rico"].loc[:, "County"] = census_agg[census_agg["State"] == "Puerto Rico"]["County"].str[:-3]

{'Guam', 'Northern Mariana Islands', 'Virgin Islands'}


c:\users\hebel\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\indexing.py:1675: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [10]:
# Merge the dataframes
merge_agg = covid_agg.join(census_agg.set_index(["State","County"]), on=["state","county"])

# Normalize Covid data per 100000 residents per county
# ((COVID total cases in county * 100000) / population of county)
merge_agg["cases"] = merge_agg["cases"]*100000/merge_agg["TotalPop"]
merge_agg["deaths"] = merge_agg["deaths"]*100000/merge_agg["TotalPop"]
merge_agg["dec2020_cases"] = merge_agg["dec2020_cases"]*100000/merge_agg["TotalPop"]
merge_agg["dec2020_deaths"] = merge_agg["dec2020_deaths"]*100000/merge_agg["TotalPop"]
merge_agg.columns=["state", "county", "norm_total_cases", "norm_total_deaths", "norm_dec20_cases", "norm_dec20_deaths", "TotalPop", "Income", "IncomePerCap", "Poverty(%)"]

# Merged Data for Oregon State
oregon = merge_agg[merge_agg["state"] == "Oregon"]
oregon

,state,county,norm_total_cases,norm_total_deaths,norm_dec20_cases,norm_dec20_deaths,TotalPop,Income,IncomePerCap,Poverty(%)
2243,Oregon,Baker,3.478473e+05,4148.936170,73141.426783,832.290363,15980.0,264799.0,16.570651,15.083855
2244,Oregon,Benton,2.042233e+05,2610.794457,38821.969654,315.017734,88249.0,972822.0,11.023604,22.421152
2245,Oregon,Clackamas,3.211310e+05,5010.475995,65458.718578,781.324226,399962.0,6185974.0,15.466404,8.976120
2246,Oregon,Clatsop,2.042713e+05,754.846006,37976.381473,123.615896,38021.0,577481.0,15.188475,12.190090
2247,Oregon,Columbia,2.097795e+05,2714.760890,42741.052045,529.806601,50207.0,585872.0,11.669130,12.315329
2248,Oregon,Coos,1.590836e+05,1540.026382,29888.272596,239.983471,62921.0,568363.0,9.032962,17.896488
2249,Oregon,Crook,2.572317e+05,5221.715707,50872.588295,902.518764,21717.0,170657.0,7.858222,15.320864
2250,Oregon,Curry,1.342673e+05,1756.267596,30124.681593,321.758949,22377.0,214926.0,9.604773,15.408656
2251,Oregon,Deschutes,2.908802e+05,2361.953217,58458.484722,321.125250,175321.0,1449064.0,8.265205,12.100898
2252,Oregon,Douglas,1.626311e+05,3702.498699,34942.738157,896.110657,107576.0,999898.0,9.294806,17.025995


In [11]:
# Analysis
# Part 1
# COVID total cases vs. % population in poverty
R = oregon["norm_total_cases"].corr(oregon["Poverty(%)"])
print(R)
# COVID total deaths vs. % population in poverty
R = oregon["norm_total_deaths"].corr(oregon["Poverty(%)"])
print(R)
# COVID total cases vs. Per Capita Income level
R = oregon["norm_total_cases"].corr(oregon["IncomePerCap"])
print(R)
# COVID total deaths vs. Per Capita Income level
R = oregon["norm_total_deaths"].corr(oregon["IncomePerCap"])
print(R)
# COVID cases during December 2020 vs. % population in poverty
R = oregon["norm_dec20_cases"].corr(oregon["Poverty(%)"])
print(R)
# COVID deaths during December 2020 vs. % population in poverty
R = oregon["norm_dec20_deaths"].corr(oregon["Poverty(%)"])
print(R)
# COVID cases during December 2020 vs. Per Capita Income level
R = oregon["norm_dec20_cases"].corr(oregon["IncomePerCap"])
print(R)
# COVID deaths during December 2020 vs. Per Capita Income level
R = oregon["norm_dec20_deaths"].corr(oregon["IncomePerCap"])

0.2870786080213774
0.3605391158241334
-0.1955172257592511
-0.20328993422383196
0.29815203013315394
0.30272695128314736
-0.2279064236544531


In [12]:
# Part 2
# COVID total cases vs. % population in poverty
R = merge_agg["norm_total_cases"].corr(merge_agg["Poverty(%)"])
print(R)
# COVID total deaths vs. % population in poverty
R = merge_agg["norm_total_deaths"].corr(merge_agg["Poverty(%)"])
print(R)
# COVID total cases vs. Per Capita Income level
R = merge_agg["norm_total_cases"].corr(merge_agg["IncomePerCap"])
print(R)
# COVID total deaths vs. Per Capita Income level
R = merge_agg["norm_total_deaths"].corr(merge_agg["IncomePerCap"])
print(R)
# COVID cases during December 2020 vs. % population in poverty
R = merge_agg["norm_dec20_cases"].corr(merge_agg["Poverty(%)"])
print(R)
# COVID deaths during December 2020 vs. % population in poverty
R = merge_agg["norm_dec20_deaths"].corr(merge_agg["Poverty(%)"])
print(R)
# COVID cases during December 2020 vs. Per Capita Income level
R = merge_agg["norm_dec20_cases"].corr(merge_agg["IncomePerCap"])
print(R)
# COVID deaths during December 2020 vs. Per Capita Income level
R = merge_agg["norm_dec20_deaths"].corr(merge_agg["IncomePerCap"])


0.2064218120793579
0.27952864246240644
-0.08858849770483875
-0.06279029278876062
0.07203914832374625
0.21320194366091585
-0.04616216166088188
